In [3]:
# !pip install convokit
# !pip install wordcloud
# !pip install --upgrade Pillow
# !pip install --upgrade pip 
# !pip install --upgrade Pillow
# !python -m pip install spacy-llm
# !python -m pip install spacy
# !pip install -U pip setuptools wheel
# !pip install -U spacy
#!python -m spacy download en_core_web_sm
# !pip install -U spacy[cuda113]
# !pip install -U spacy[cuda90]
# !pip install spacy_transformers
# !pip install accelerate
# !python -m spacy download en_core_web_trf

## Importar librerías

In [2]:
# Bloque 1: Librerías básicas de manipulación y procesamiento de datos
import re, os
import math
import pandas as pd
import numpy as np
import nltk
import json

# Bloque 2: Librerías para procesamiento de texto y visualización
from nltk.corpus import stopwords
from wordcloud import WordCloud

# Bloque 3: Librerías para preprocesamiento y evaluación de modelos de machine learning
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

# Bloque 4: Procesamiento de texto con spaCy
from collections import Counter
import spacy
from spacy import displacy
spacy.prefer_gpu()
nlp = spacy.load("en_core_web_sm")

# Bloque 5: Librerías de visualización gráfica
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

# Bloque 6: Configuración de TensorFlow y Keras para aprendizaje profundo
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, GRU, Dense, Embedding
from keras.preprocessing.text import Tokenizer

# Bloque 7: Librerías para manejo de modelos de lenguaje y transferencia de conocimiento
import tensorflow_hub as hub
from huggingface_hub import hf_hub_download
from transformers import AutoModelForTokenClassification, pipeline
from accelerate import Accelerator

# Bloque 8: Librerías para evaluación de modelos y métricas
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix, recall_score, precision_score, ConfusionMatrixDisplay, fbeta_score, f1_score
from sklearn.metrics import RocCurveDisplay
from sklearn import metrics


# Bloque 9: Librerías para procesamiento de texto con Transformers y Hugging Face
import plotly.express as px
import spacy


# Bloque 10: Librerías para análisis de similitud y extracción de características de texto
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS



## Funciones

In [ ]:
def remove_nan_rows(dataframe, columns=['messages', 'sender_labels', 'receiver_labels']):
    """
    Elimina las filas con valores NaN en columnas especificadas.

    Parámetros:
    - dataframe: pd.DataFrame
        El DataFrame a procesar.
    - columns: list
        Lista de columnas en las que buscar valores NaN.

    Retorna:
    - pd.DataFrame
        DataFrame con las filas que contienen valores NaN eliminadas.
    """
    return dataframe.dropna(subset=columns)


def remove_tags(string, remove_special_chars=False, remove_stopwords=False, remove_newlines=False):
    """
    Elimina etiquetas y contenido no deseado de una cadena de texto, como etiquetas HTML, nombres de usuario de Twitter, hashtags, números y URLs.

    Parámetros:
    - string: str
        Cadena de texto a procesar.
    - remove_special_chars: bool
        Indica si se deben eliminar caracteres no alfanuméricos.
    - remove_stopwords: bool
        Indica si se deben eliminar palabras comunes (stopwords).
    - remove_newlines: bool
        Indica si se deben eliminar caracteres de nueva línea.

    Retorna:
    - str
        Cadena de texto procesada y limpia.
    """
    result = re.sub(r'<.*?>', '', string)  # Elimina etiquetas HTML
    result = re.sub('@[\w]+', '', result)  # Elimina nombres de usuario de Twitter
    result = re.sub('#[\w]+', '', result)  # Elimina hashtags
    result = re.sub("\d+", " ", result)  # Elimina números
    result = re.sub(r'http\S+', '', result)  # Elimina URLs

    if remove_special_chars:
        result = re.sub(r'[^\w\s]', ' ', result)  # Elimina caracteres no alfanuméricos

    if remove_newlines:
        result = re.sub(r'\n\n', ' ', result)  # Elimina caracteres de nueva línea
        result = ' '.join(result.split())  # Divide y une para eliminar espacios adicionales

    if remove_stopwords:
        stop_words = set(stopwords.words('english'))
        result = ' '.join([w for w in result.split() if w.lower() not in stop_words])

    # Elimina palabras de longitud 1
    result = ' '.join([word for word in result.split() if len(word) > 1])

    result = result.lower()
    return result


def compute_class_weight(train_y):
    """
    Calcula el peso de las clases dado un conjunto de datos de entrenamiento desbalanceado.
    Generalmente utilizado en modelos de redes neuronales para ajustar la función de pérdida (función de pérdida ponderada),
    dando más peso a las clases raras.

    Parámetros:
    - train_y: array
        Etiquetas del conjunto de datos de entrenamiento.

    Retorna:
    - dict
        Diccionario que asigna el peso a cada clase.
    """
    import sklearn.utils.class_weight as scikit_class_weight

    class_list = list(set(train_y))
    class_weight_value = scikit_class_weight.compute_class_weight(class_weight='balanced', classes=class_list, y=train_y)
    class_weight = dict()

    # Inicializa todas las clases en el diccionario con peso 1
    curr_max = int(np.max(class_list))
    for i in range(curr_max):
        class_weight[i] = 1

    # Construye el diccionario utilizando el peso obtenido de la función de scikit
    for i in range(len(class_list)):
        class_weight[class_list[i]] = class_weight_value[i]

    return class_weight


## Importar y Preparar Datos

In [ ]:
# Paso 1: Cargar datos de entrenamiento desde el archivo 'train.jsonl'
data_list = []
messages = []
with open('NLP_Diplomacy/train.jsonl', 'r') as archivo:
    for line in archivo:
        try:
            data = json.loads(line)
            data_list.append(data)
            messages.extend(data['messages'])
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")

# Paso 2: Cargar datos de validación desde el archivo 'validation.jsonl'
validation_list = []
messages = []
with open('NLP_Diplomacy/validation.jsonl', 'r') as archivo:
    for line in archivo:
        try:
            validation = json.loads(line)
            validation_list.append(validation)
            messages.extend(validation['messages'])
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")

# Paso 3: Cargar datos de prueba desde el archivo 'test.jsonl'
test_list = []
messages = []
with open('NLP_Diplomacy/test.jsonl', 'r') as archivo:
    for line in archivo:
        try:
            test = json.loads(line)
            test_list.append(validation)
            messages.extend(validation['messages'])
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")

# Paso 4: Crear DataFrames a partir de las listas de datos
df = pd.DataFrame(data_list)
df_val = pd.DataFrame(validation_list)
df_test = pd.DataFrame(test_list)

# Paso 5: "Explotar" las columnas de listas en filas individuales
df_explode = df.explode(['messages', 'sender_labels', 'receiver_labels', 'speakers', 'receivers', 'absolute_message_index', 'relative_message_index', 'seasons', 'years', 'game_score', 'game_score_delta'], ignore_index=True)
df_val = df_val.explode(['messages', 'sender_labels', 'receiver_labels', 'speakers', 'receivers', 'absolute_message_index', 'relative_message_index', 'seasons', 'years', 'game_score', 'game_score_delta'], ignore_index=True)
df_test = df_test.explode(['messages', 'sender_labels', 'receiver_labels', 'speakers', 'receivers', 'absolute_message_index', 'relative_message_index', 'seasons', 'years', 'game_score', 'game_score_delta'], ignore_index=True)

# Paso 6: Limpiar filas con valores NaN
df_explode_cleaned = remove_nan_rows(df_explode)
df_val_cleaned = remove_nan_rows(df_val)
df_test_cleaned = remove_nan_rows(df_test)

# Paso 7: Convertir la columna 'messages' a tipo de dato string y limpiar el texto
df_explode_cleaned['messages'] = df_explode_cleaned['messages'].astype(str)
df_explode_cleaned.loc[:, 'messages_clean'] = df_explode_cleaned['messages'].apply(lambda cw: remove_tags(cw, remove_special_chars=True, remove_stopwords=True, remove_newlines=True))

df_val_cleaned['messages'] = df_val_cleaned['messages'].astype(str)
df_val_cleaned.loc[:, 'messages_clean'] = df_val_cleaned['messages'].apply(lambda cw: remove_tags(cw, remove_special_chars=True, remove_stopwords=True, remove_newlines=True))

df_test_cleaned['messages'] = df_test_cleaned['messages'].astype(str)
df_test_cleaned.loc[:, 'messages_clean'] = df_test_cleaned['messages'].apply(lambda cw: remove_tags(cw, remove_special_chars=True, remove_stopwords=True, remove_newlines=True))

# Paso 8: Limpiar filas con valores NaN después de la limpieza adicional
df_explode_cleaned = remove_nan_rows(df_explode_cleaned)
df_val_cleaned = remove_nan_rows(df_val_cleaned)
df_test_cleaned = remove_nan_rows(df_test_cleaned)

# Paso 9: Calcular la longitud del texto en cada fila
df_explode_cleaned['caracteres_texto'] = df_explode_cleaned['messages_clean'].apply(lambda row: len(str(row)) if not pd.isna(row) else np.nan)

# Paso 10: Filtrar DataFrame con condiciones específicas
filtered_dfb = df_explode_cleaned[(df_explode_cleaned['sender_labels'] == False) & (df_explode_cleaned['receiver_labels'] == True) | 
                                  (df_explode_cleaned['sender_labels'] == True) & (df_explode_cleaned['receiver_labels'] == False)| 
                                  (df_explode_cleaned['sender_labels'] == False) & (df_explode_cleaned['receiver_labels'] == False)| 
                                  (df_explode_cleaned['sender_labels'] == True) & (df_explode_cleaned['receiver_labels'] == True) & (df_explode_cleaned['caracteres_texto'] <= 20)]



## Preparando Datos para el modelo

In [61]:
# Paso 11: Obtener mensajes de entrenamiento y etiquetas asociadas
train_messages = filtered_dfb['messages_clean'].values
train_labels = filtered_dfb["sender_labels"].values

# Paso 12: Obtener mensajes de validación y etiquetas asociadas
val_messages = df_val_cleaned['messages_clean'].values
val_labels = df_val_cleaned["sender_labels"].values

# Paso 13: Obtener mensajes de prueba y etiquetas asociadas
test_messages = df_test_cleaned['messages_clean'].values
test_labels = df_test_cleaned["sender_labels"].values

# Paso 14: Inicializar y ajustar el codificador de etiquetas
encoder = LabelEncoder()

# Paso 15: Codificar las etiquetas para entrenamiento, validación y prueba
train_encoded_labels = encoder.fit_transform(train_labels)
val_encoded_labels = encoder.fit_transform(val_labels)
test_encoded_labels = encoder.fit_transform(test_labels)


In [64]:
# Tokenización y creación del diccionario de palabras (Tokens) restringiendo la cantidad de palabras
vocab_size = 1000 ## -> Parámetro a calibrar
oov_tok = ''
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_messages)
word_index = tokenizer.word_index

## Preparando Embeddings

In [65]:
#Padding
max_length = 30 ## -> Parámetro a calibrar
padding_type='post'
trunc_type='post'
# Se convierte la secuencia de train
train_sequences = tokenizer.texts_to_sequences(train_messages)
train_padded = tf.keras.preprocessing.sequence.pad_sequences(train_sequences, padding='post', maxlen=max_length)
# Se convierte la secuencia de val
val_sequences = tokenizer.texts_to_sequences(val_messages)
val_padded = tf.keras.preprocessing.sequence.pad_sequences(val_sequences, padding='post', maxlen=max_length)
# Se convierte la secuencia de test
test_sequences = tokenizer.texts_to_sequences(test_messages)
test_padded = tf.keras.preprocessing.sequence.pad_sequences(test_sequences, padding='post', maxlen=max_length)

## Modelado

### Definición y Documentación del Modelo GRU

In [66]:
embedding_dim = 32

gru_model = Sequential(name="GRU_Model")
gru_model.add(Embedding(vocab_size,embedding_dim,input_length=max_length))
gru_model.add(GRU(128))
gru_model.add(Dense(1, activation='sigmoid'))
 
print(gru_model.summary())

optimizador = tf.keras.optimizers.Adam(learning_rate=0.0005,
                                       beta_1=0.9,
                                       beta_2=0.999,
                                       epsilon=1e-07)
gru_model.compile(loss='binary_crossentropy',optimizer="nadam",metrics=['accuracy'])

Model: "GRU_Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 30, 32)            32000     
                                                                 
 gru (GRU)                   (None, 128)               62208     
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 94,337
Trainable params: 94,337
Non-trainable params: 0
_________________________________________________________________
None


In [67]:
history = gru_model.fit(train_padded,
                        train_encoded_labels,batch_size=32,
                        epochs=7,
                        verbose=1,
                        validation_data=(val_padded, val_encoded_labels))
 
# Se imprime el valor de la función de pérdida y accuracy sobre el set de Test
print()
print("GRU model Score---> ", gru_model.evaluate(test_padded, test_encoded_labels, verbose=0))

Epoch 1/7
126/126 [==============================] - 4s 18ms/step - loss: 0.3578 - accuracy: 0.8644 - val_loss: 0.7396 - val_accuracy: 0.3001
Epoch 2/7
126/126 [==============================] - 2s 17ms/step - loss: 0.2637 - accuracy: 0.8814 - val_loss: 0.5405 - val_accuracy: 0.5148
Epoch 3/7
126/126 [==============================] - 2s 19ms/step - loss: 0.2479 - accuracy: 0.8939 - val_loss: 0.4753 - val_accuracy: 0.8969
Epoch 4/7
126/126 [==============================] - 2s 19ms/step - loss: 0.2331 - accuracy: 0.9026 - val_loss: 0.4891 - val_accuracy: 0.5593
Epoch 5/7
126/126 [==============================] - 2s 19ms/step - loss: 0.2259 - accuracy: 0.9096 - val_loss: 0.5869 - val_accuracy: 0.5191
Epoch 6/7
126/126 [==============================] - 2s 19ms/step - loss: 0.2179 - accuracy: 0.9131 - val_loss: 0.4456 - val_accuracy: 0.6179
Epoch 7/7
126/126 [==============================] - 2s 16ms/step - loss: 0.2062 - accuracy: 0.9236 - val_loss: 0.3509 - val_accuracy: 0.7620

GRU m

#### Explicación Conceptual:
Este modelo utiliza una arquitectura de Red Neuronal Recurrente (RNN) tipo GRU (Gated Recurrent Unit) para el procesamiento de secuencias de texto. La capa de Embedding convierte las palabras en vectores de números reales. La capa GRU ayuda al modelo a aprender patrones secuenciales en los datos de entrada. Finalmente, la capa Dense con función de activación sigmoidal realiza la clasificación binaria. El modelo se compila con una función de pérdida de entropía cruzada binaria y se entrena con datos de entrenamiento, validación y se evalúa en un conjunto de prueba. Los resultados incluyen la precisión y la función de pérdida en el conjunto de prueba.

Descripción:

Este modelo utiliza una capa de Embedding para convertir las palabras en vectores de longitud fija.
La capa GRU (Gated Recurrent Unit) es una red neuronal recurrente que ayuda a modelar dependencias a largo plazo en secuencias de datos.
La capa Dense con activación sigmoide produce la salida binaria, indicando la probabilidad de que la etiqueta sea 1 (True).

Resultados:

El modelo muestra una precisión (accuracy) de alrededor del 92.36% en el conjunto de entrenamiento y del 76.20% en el conjunto de validación después de 7 épocas.
La evaluación en el conjunto de prueba indica una pérdida de aproximadamente 0.3081 y una precisión del 86.67%.
Análisis:

El modelo parece tener un buen rendimiento en el conjunto de entrenamiento, pero hay una diferencia significativa en la precisión entre el conjunto de entrenamiento y el conjunto de validación, lo que sugiere cierto grado de sobreajuste.
Sería recomendable ajustar la arquitectura del modelo o considerar técnicas de regularización para mejorar la generalización en el conjunto de validación y prueba.
Se podría probar con diferentes valores de embedding_dim y max_length para encontrar la configuración óptima del modelo.



## Modelo con Transfer Learning (Universal Sentence Encoder):

Descripción:

Este modelo utiliza transfer learning con el Universal Sentence Encoder (USE) como capa inicial.
Se utiliza el paquete tensorflow_hub para cargar el modelo USE directamente desde TensorFlow Hub.
La capa USE se ajusta para ser trainable (trainable=True) en este caso.
Se agrega una capa de Reshape para ajustar la forma de entrada al modelo.
Se añade una capa GRU (Gated Recurrent Unit) con 128 unidades para procesar las secuencias.
La capa Dense con activación sigmoide produce la salida con dos clases (binaria).
Transfer Learning:

El Universal Sentence Encoder es un modelo de procesamiento de lenguaje natural preentrenado que captura información semántica de las oraciones.
En este caso, la capa USE se utiliza como un extractor de características y se conecta a las capas adicionales del modelo para la tarea específica de clasificación binaria.
Al ajustar trainable=True, se permite que las capas del USE se adapten a los datos específicos de la tarea.

In [69]:
train_labels_one_hot = tf.keras.utils.to_categorical(train_labels, num_classes=2)
test_labels_one_hot = tf.keras.utils.to_categorical(test_labels, num_classes=2)
val_labels_one_hot = tf.keras.utils.to_categorical(val_labels, num_classes=2)
train_labels_one_hot


array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [71]:
os.environ["TFHUB_CACHE_DIR"] = "my_tfhub_cache"
model= tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                   trainable=True, dtype=tf.string, input_shape=[]),
    tf.keras.layers.Reshape(target_shape=(1, 512)),  # Ajustar la forma de entrada
    tf.keras.layers.GRU(128),
    tf.keras.layers.Dense(2, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer="nadam",
metrics=["accuracy"])


history = model.fit(train_messages, train_labels_one_hot, batch_size=32, epochs=7, validation_data=(val_messages, val_labels_one_hot))

print()
print("Model Score: ", model.evaluate(test_messages, test_labels_one_hot, verbose=0))


Epoch 1/7
126/126 [==============================] - 180s 1s/step - loss: 0.3079 - accuracy: 0.8786 - val_loss: 0.3823 - val_accuracy: 0.8220
Epoch 2/7
126/126 [==============================] - 179s 1s/step - loss: 0.2374 - accuracy: 0.9013 - val_loss: 0.2915 - val_accuracy: 0.9174
Epoch 3/7
126/126 [==============================] - 202s 2s/step - loss: 0.1642 - accuracy: 0.9463 - val_loss: 0.5463 - val_accuracy: 0.7196
Epoch 4/7
126/126 [==============================] - 185s 1s/step - loss: 0.0859 - accuracy: 0.9773 - val_loss: 1.1227 - val_accuracy: 0.6109
Epoch 5/7
126/126 [==============================] - 174s 1s/step - loss: 0.0512 - accuracy: 0.9865 - val_loss: 1.0344 - val_accuracy: 0.6773
Epoch 6/7
126/126 [==============================] - 191s 2s/step - loss: 0.0402 - accuracy: 0.9890 - val_loss: 0.8461 - val_accuracy: 0.7535
Epoch 7/7
126/126 [==============================] - 181s 1s/step - loss: 0.0342 - accuracy: 0.9895 - val_loss: 1.3460 - val_accuracy: 0.6582

Model

El modelo muestra un rendimiento en la tarea de clasificación binaria.
La precisión en el conjunto de prueba se puede verificar con model.evaluate.
El uso del Universal Sentence Encoder permite aprovechar la información semántica preentrenada y mejorar el rendimiento en tareas específicas.

1. Rendimiento durante el Entrenamiento:

Precisión (Accuracy): La precisión del modelo en el conjunto de entrenamiento aumenta progresivamente, alcanzando un valor de aproximadamente 98.95% al final del entrenamiento.
Pérdida (Loss): La función de pérdida disminuye, indicando que el modelo está aprendiendo y mejorando su capacidad para hacer predicciones precisas.

2. Evaluación en el Conjunto de Validación:

La precisión en el conjunto de validación muestra variabilidad, alcanzando un máximo de aproximadamente 91.74% en la tercera época. Posteriormente, la precisión disminuye en las últimas épocas.
La variabilidad y la disminución de la precisión podrían indicar cierta sensibilidad a overfitting o la necesidad de ajustar hiperparámetros.

3. Model Score en el Conjunto de Prueba:

El modelo logra una precisión del 80% en el conjunto de prueba según el model.evaluate.
La pérdida en el conjunto de prueba es de aproximadamente 0.419.
Consideraciones:

El rendimiento del modelo en el conjunto de prueba es razonable, pero podría haber espacio para mejorar.
La disminución en la precisión después de las primeras épocas en el conjunto de validación podría indicar overfitting. Se podría considerar la incorporación de técnicas de regularización o ajuste de la tasa de aprendizaje.
Es importante considerar el equilibrio entre la precisión en el conjunto de entrenamiento y la capacidad del modelo para generalizar a nuevos datos.
Se podría explorar la posibilidad de ajustar la arquitectura del modelo, experimentar con diferentes tasas de aprendizaje o aplicar técnicas de regularización para mejorar aún más el rendimiento del modelo.

## Nombre del Modelo: Universal Sentence Encoder GRU

Descripción: Este modelo utiliza el Universal Sentence Encoder (versión 4) como una capa de entrada para la representación semántica de oraciones. Luego, se agrega una capa de Reshape para ajustar la forma de entrada antes de pasar a una capa GRU (Gated Recurrent Unit) con 128 unidades. Finalmente, hay una capa densa con función de activación sigmoide para la clasificación binaria.

Configuración Adicional:

Optimizador: Adam con tasa de aprendizaje inicial de 0.01, decaimiento de 0.004, y otros parámetros configurados.
Función de Pérdida: Binary Crossentropy.
Métrica de Evaluación: Precisión (accuracy).

In [72]:
os.environ["TFHUB_CACHE_DIR"] = "my_tfhub_cache"
model11= tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                   trainable=True, dtype=tf.string, input_shape=[]),
    tf.keras.layers.Reshape(target_shape=(1, 512)),  # Ajustar la forma de entrada
    tf.keras.layers.GRU(128),
    tf.keras.layers.Dense(2, activation='sigmoid')
])
optimizador = tf.keras.optimizers.Adam(learning_rate=0.01,
                                       decay=0.004,
                                       beta_1=0.9,
                                       beta_2=0.999,
                                       epsilon=1e-07)
model11.compile(loss='binary_crossentropy', optimizer=optimizador,
metrics=["accuracy"])


history11 = model11.fit(train_messages, train_labels_one_hot, batch_size=32, epochs=7, validation_data=(val_messages, val_labels_one_hot))

print()
print("Model Score: ", model11.evaluate(test_messages, test_labels_one_hot, verbose=0))

Epoch 1/7
126/126 [==============================] - 297s 2s/step - loss: 0.2982 - accuracy: 0.8734 - val_loss: 0.5263 - val_accuracy: 0.5261
Epoch 2/7
126/126 [==============================] - 264s 2s/step - loss: 0.1360 - accuracy: 0.9593 - val_loss: 0.6439 - val_accuracy: 0.7429
Epoch 3/7
126/126 [==============================] - 282s 2s/step - loss: 0.0577 - accuracy: 0.9868 - val_loss: 1.0636 - val_accuracy: 0.6836
Epoch 4/7
126/126 [==============================] - 281s 2s/step - loss: 0.0388 - accuracy: 0.9903 - val_loss: 1.0702 - val_accuracy: 0.6737
Epoch 5/7
126/126 [==============================] - 283s 2s/step - loss: 0.0343 - accuracy: 0.9908 - val_loss: 2.3124 - val_accuracy: 0.5240
Epoch 6/7
126/126 [==============================] - 290s 2s/step - loss: 0.0318 - accuracy: 0.9920 - val_loss: 1.6381 - val_accuracy: 0.6257
Epoch 7/7
126/126 [==============================] - 299s 2s/step - loss: 0.0298 - accuracy: 0.9918 - val_loss: 2.2865 - val_accuracy: 0.5685

Model

Resultados del Entrenamiento:

Precisión en Entrenamiento: Aumenta progresivamente y alcanza alrededor del 99.18% al final del entrenamiento.
Pérdida en Entrenamiento: Disminuye, indicando aprendizaje.
Precisión en Validación: Alcanza alrededor del 68.36% en la tercera época pero muestra variabilidad y disminución en las últimas épocas.
Pérdida en Validación: Aumenta, indicando posible overfitting.
Model Score en el Conjunto de Prueba:

Precisión en Prueba: 80%
Pérdida en Prueba: 0.597
Análisis de Resultados:

La precisión en entrenamiento es alta, pero hay signos de overfitting, ya que la precisión en validación varía y disminuye en las últimas épocas.
La pérdida en validación aumenta, lo que sugiere que el modelo podría estar memorizando en lugar de generalizar.
La precisión en el conjunto de prueba es razonable, pero podría haber espacio para mejorar.
Se podría explorar la posibilidad de ajustar la arquitectura del modelo, experimentar con diferentes tasas de aprendizaje o aplicar técnicas de regularización para mejorar aún más el rendimiento del modelo.

## Guardar lo modelos

In [73]:
gru_model.save('gru_model.h5', overwrite=True)
model.save('model.h5', overwrite=True)
model11.save('model11.h5', overwrite=True)

## Carga de los modelos

In [ ]:
gru_model = tf.keras.models.load_model('gru_model.h5')
model = tf.keras.models.load_model('model.h5')
model11 = tf.keras.models.load_model('model11.h5')

## Evaluacion de los modelos

In [74]:
# Tamaño del lote
lote = 32

# Evaluar el modelo en el conjunto de prueba
evaluacion11 = model11.evaluate(test_messages, test_labels_one_hot,
                                batch_size=lote,
                                verbose="auto")

# Imprimir los resultados de la evaluación
print('\n Resultados de la evaluación del modelo_', evaluacion11)

20/20 [==============================] - 0s 6ms/step - loss: 0.5968 - accuracy: 0.8000

 Resultados de la evaluación del modelo_ [0.5967699289321899, 0.800000011920929]


### Prediccion del Modelo: Universal Sentence Encoder GRU

In [75]:
# Predicciones sobre datos test
# Convierte los datos de entrada en una lista de cadenas
test_messages = [str(message) for message in test_messages]

# Predicciones sobre datos test
predicciones = model11.predict(test_messages)

predicciones_lista = []
for i in predicciones:
    predicciones_lista.append(np.argmax(i))
predicciones_lista = np.asarray(predicciones_lista)

20/20 [==============================] - 4s 5ms/step


In [76]:
predicciones_lista

array([1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1,
       1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1,
       1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1,
       1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0,
       1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0,
       1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0,
       0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1,
       0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1,
       0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0,
       1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1,

In [78]:


def calculate_metrics(model, messages, labels, beta=1):
    # Convertir mensajes a una lista de cadenas
    messages = [str(message) for message in messages]
    
    # Obtener predicciones del modelo
    predictions = model.predict(messages)
    
    # Convertir predicciones a etiquetas binarias
    predicted_labels = np.argmax(predictions, axis=1)
    
    # Convertir etiquetas verdaderas a etiquetas binarias si es necesario
    true_labels = np.argmax(labels, axis=1) if len(labels.shape) > 1 else labels
    
    # Calcular matriz de confusión
    confusion_mat = confusion_matrix(true_labels, predicted_labels)
    
    # Calcular F1 Score
    f1 = f1_score(true_labels, predicted_labels, average='weighted')
    
    # Calcular Fbeta Score (con beta específico, por defecto beta=1 para F1 Score)
    fbeta = fbeta_score(true_labels, predicted_labels, beta=beta, average='weighted')
    
    # Calcular Precision y Recall
    precision = precision_score(true_labels, predicted_labels, average='weighted')
    recall = recall_score(true_labels, predicted_labels, average='weighted')
    
    return confusion_mat, f1, fbeta, precision, recall

conf_mat, f1_scr, fbeta_scr, precision_scr, recall_scr = calculate_metrics(model11, test_messages, test_encoded_labels)
print("Confusion Matrix:\n", conf_mat)
print("F1 Score:", f1_scr)
print("Fbeta Score:", fbeta_scr)
print("Precision:", precision_scr)
print("Recall:", recall_scr)


20/20 [==============================] - 0s 5ms/step
Confusion Matrix:
 [[ 84  42]
 [ 84 420]]
F1 Score: 0.8099378881987578
Fbeta Score: 0.8099378881987578
Precision: 0.8272727272727274
Recall: 0.8


### Prediccion del Modelo con Transfer Learning (Universal Sentence Encoder)


In [79]:
# Predicciones sobre datos test
# Convierte los datos de entrada en una lista de cadenas
test_messages = [str(message) for message in test_messages]

# Predicciones sobre datos test
predicciones = model.predict(test_messages)

predicciones_lista = []
for i in predicciones:
    predicciones_lista.append(np.argmax(i))
predicciones_lista = np.asarray(predicciones_lista)

20/20 [==============================] - 1s 5ms/step


In [80]:


conf_mat, f1_scr, fbeta_scr, precision_scr, recall_scr = calculate_metrics(model, test_messages, test_encoded_labels)
print("Confusion Matrix:\n", conf_mat)
print("F1 Score:", f1_scr)
print("Fbeta Score:", fbeta_scr)
print("Precision:", precision_scr)
print("Recall:", recall_scr)

20/20 [==============================] - 0s 6ms/step
Confusion Matrix:
 [[ 84  42]
 [ 84 420]]
F1 Score: 0.8099378881987578
Fbeta Score: 0.8099378881987578
Precision: 0.8272727272727274
Recall: 0.8


Modelo Benchmark

Modelo Benchmark muestra los resultados de un estudio sobre el uso de diferentes modelos para detectar mentiras. El estudio se dividió en dos tareas: detectar mentiras reales y detectar mentiras sospechosas.

En la tarea de detectar mentiras reales, el modelo más eficaz fue el modelo neural que integraba mensajes anteriores y dinámicas de poder. Este modelo obtuvo una puntuación F1 de 56,1, que es comparable a la puntuación F1 de un humano.

En la tarea de detectar mentiras sospechosas, el modelo más eficaz fue el modelo de bag of words. Este modelo obtuvo una puntuación F1 de 51,6, que es ligeramente superior a la puntuación F1 del modelo de mayoría de clases.

En general, los resultados del estudio sugieren que los modelos neurales son más eficaces para detectar mentiras reales que los modelos basados en características lingüísticas. Sin embargo, los modelos basados en características lingüísticas son más eficaces para detectar mentiras sospechosas.

Aquí hay una interpretación más detallada de los resultados:

Tarea de detectar mentiras reales

El modelo aleatorio obtuvo una puntuación F1 de 39,8. Esto significa que, en promedio, el modelo aleatorio acertó en el 39,8% de los casos.
El modelo de mayoría de clases obtuvo una puntuación F1 de 47,8. Esto significa que, en promedio, el modelo de mayoría de clases acertó en el 47,8% de los casos.
El modelo de harbingers obtuvo una puntuación F1 de 52,8. Esto significa que, en promedio, el modelo de harbingers acertó en el 52,8% de los casos.
El modelo de bag of words obtuvo una puntuación F1 de 54,3. Esto significa que, en promedio, el modelo de bag of words acertó en el 54,3% de los casos.
El modelo de LSTM obtuvo una puntuación F1 de 53,8. Esto significa que, en promedio, el modelo de LSTM acertó en el 53,8% de los casos.
El modelo de contexto LSTM obtuvo una puntuación F1 de 55,8. Esto significa que, en promedio, el modelo de contexto LSTM acertó en el 55,8% de los casos.
El modelo de contexto LSTM + BERT obtuvo una puntuación F1 de 56,1. Esto significa que, en promedio, el modelo de contexto LSTM + BERT acertó en el 56,1% de los casos.
Como se puede ver, el modelo de contexto LSTM + BERT obtuvo la puntuación F1 más alta, lo que lo convierte en el modelo más eficaz para detectar mentiras reales. Este modelo integra mensajes anteriores y dinámicas de poder, lo que le permite identificar patrones que los otros modelos no pueden ver.

Tarea de detectar mentiras sospechosas

El modelo aleatorio obtuvo una puntuación F1 de 38,3. Esto significa que, en promedio, el modelo aleatorio acertó en el 38,3% de los casos.
El modelo de mayoría de clases obtuvo una puntuación F1 de 48,3. Esto significa que, en promedio, el modelo de mayoría de clases acertó en el 48,3% de los casos.
El modelo de harbingers obtuvo una puntuación F1 de 45,9. Esto significa que, en promedio, el modelo de harbingers acertó en el 45,9% de los casos.
El modelo de bag of words obtuvo una puntuación F1 de 51,5. Esto significa que, en promedio, el modelo de bag of words acertó en el 51,5% de los casos.
El modelo de LSTM obtuvo una puntuación F1 de 53,8. Esto significa que, en promedio, el modelo de LSTM acertó en el 53,8% de los casos.
Como se puede ver, el modelo de bag of words obtuvo la puntuación F1 más alta, lo que lo convierte en el modelo más eficaz para detectar mentiras sospechosas. Este modelo utiliza características lingüísticas para identificar patrones que pueden indicar que una persona está mintiendo.

En general, los resultados del estudio sugieren que los modelos neurales son más eficaces para detectar mentiras reales que los modelos basados en características lingüísticas. Sin embargo, los modelos basados en características lingüísticas son más eficaces para detectar mentiras sospechosas.

#### Comparacion de los resultados con el modelo:

Los resultados de los dos modelos que se presentan son muy similares a los resultados del estudio que se muestra en la imagen. En ambos casos, los modelos neurales obtienen una puntuación F1 más alta en la tarea de detectar mentiras reales que en la tarea de detectar mentiras sospechosas.

En particular, el modelo Universal Sentence Encoder GRU obtuvo una puntuación F1 de 0,8099 en ambas tareas. Este modelo utiliza una arquitectura GRU para procesar el texto, lo que le permite aprender patrones a nivel de palabras y oraciones.

El modelo Universal Sentence Encoder con Transfer Learning también obtuvo una puntuación F1 de 0,8099 en ambas tareas. Este modelo utiliza un modelo preentrenado de Universal Sentence Encoder para aprender las características del lenguaje.

En general, los resultados de estos dos modelos sugieren que los modelos neurales son una buena opción para detectar mentiras. Estos modelos son capaces de aprender patrones complejos en el lenguaje que pueden ayudar a identificar a las personas que están mintiendo.

Sin embargo, es importante tener en cuenta que los resultados de estos estudios se basan en conjuntos de datos específicos. Es posible que los resultados sean diferentes si se utilizan otros conjuntos de datos. Además, los modelos neurales pueden ser sensibles a los sesgos en los datos de entrenamiento.

A continuación se presenta un análisis más detallado de los resultados de los dos modelos:

Modelo Universal Sentence Encoder GRU

Tarea de detectar mentiras reales:
Precisión: 0,8272727272727274
Recall: 0,8
F1 Score: 0,8099378881987578
El modelo Universal Sentence Encoder GRU obtuvo una precisión de 0,8272727272727274 en la tarea de detectar mentiras reales. Esto significa que el modelo acertó en el 82,72% de los casos en que una persona estaba mintiendo. El modelo también obtuvo un recall de 0,8, lo que significa que detectó el 80% de las mentiras reales.

En general, el modelo Universal Sentence Encoder GRU es capaz de detectar mentiras reales con un alto grado de precisión y recall.

Modelo Universal Sentence Encoder con Transfer Learning

Tarea de detectar mentiras reales:
Precisión: 0,8272727272727274
Recall: 0,8
F1 Score: 0,8099378881987578
El modelo Universal Sentence Encoder con Transfer Learning obtuvo los mismos resultados que el modelo Universal Sentence Encoder GRU en la tarea de detectar mentiras reales. Esto sugiere que el uso de transferencia de aprendizaje no mejora el rendimiento del modelo en esta tarea.

En general, los resultados de estos dos modelos sugieren que los modelos neurales son una buena opción para detectar mentiras. Estos modelos son capaces de aprender patrones complejos en el lenguaje que pueden ayudar a identificar a las personas que están mintiendo.